In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.sparse import csr_matrix
import tensorflow as tf
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans


In [2]:
def extract_features(images):
    # Flatten each image to a 1D array (assuming images are in shape (n_samples, height, width))
    return np.array([image.flatten() for image in images])

In [3]:
def select_anchor_points(features, n_anchors):
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=n_anchors, random_state=42)
    kmeans.fit(features)
    
    # Get the centroids of the clusters
    anchor_points = kmeans.cluster_centers_

    # Optionally, find the closest samples to each centroid to get their indices
    distances = np.linalg.norm(features[:, np.newaxis] - anchor_points, axis=2)
    indices = np.argmin(distances, axis=0)  # Get the indices of the closest points
    
    return anchor_points, indices  # Return the anchor points and their indices

In [4]:
def create_combined_adjacency_matrix(train_features, test_features, n_neighbors=5):
    # Combine train and test features
    combined_features = np.vstack((train_features, test_features))

    # Fit KNN to find nearest neighbors
    nbrs = NearestNeighbors(n_neighbors=n_neighbors + 1, metric='euclidean').fit(combined_features)
    distances, indices = nbrs.kneighbors(combined_features)

    # Create a sparse adjacency matrix
    n_samples = combined_features.shape[0]
    A = np.zeros((n_samples, n_samples))

    # Fill the adjacency matrix
    for i in range(n_samples):
        for j in range(1, n_neighbors + 1):  # Start from 1 to ignore self-loops
            if indices[i, j] < n_samples:  # Check if the index is within bounds
                A[i, indices[i, j]] = 1  # Using binary adjacency (1 for neighbors)

    return csr_matrix(A)  # Returning as sparse matrix

In [5]:
def generate_hash_codes(adjacency_matrix, n_bits=32):
    # Compute the degree matrix
    D = np.diag(np.array(adjacency_matrix.sum(axis=1)).flatten())

    # Compute the graph Laplacian
    L = D - adjacency_matrix.toarray()  # Convert to dense for Laplacian calculation

    # Compute eigenvalues and eigenvectors
    _, eigenvectors = np.linalg.eigh(L)  # Using numpy's eigenvalue decomposition

    # Use the first n_bits eigenvectors to create hash codes
    hash_codes = (eigenvectors[:, :n_bits] > 0).astype(int)  # Binarize eigenvectors
    return hash_codes

In [6]:
def image_retrieval_pipeline(train_images, test_images, n_anchors=10, n_bits=32, n_neighbors=5):
    # Step 1: Extract features from train and test images
    train_features = extract_features(train_images)
    test_features = extract_features(test_images)
    print("features extracted")
    # Step 2: Select anchor points from the training set
    anchors, anchor_indices = select_anchor_points(train_features, n_anchors)
    print("anchor points done")
    # Step 3: Create the combined adjacency matrix
    adjacency_matrix = create_combined_adjacency_matrix(train_features, test_features, n_neighbors)
    print("anchor points generated")
    # Step 4: Generate hash codes from the adjacency matrix
    hash_codes = generate_hash_codes(adjacency_matrix, n_bits)
    print("DONE :))) PLS FUCKING WORK")
    return hash_codes, anchors, anchor_indices

In [7]:
# Example usage (assuming you have train_images and test_images as numpy arrays)
n_train,n_test = 10000,10000
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data() 
x_train,y_train = x_train[0:n_train],y_train[0:n_train]
x_test,y_test = x_test[0:n_test],y_test[0:n_test]

train_images = x_train  # Load or generate your training images
test_images = x_test   # Load or generate your test images

hash_codes, anchors, anchor_indices = image_retrieval_pipeline(train_images, test_images)
print("Generated Hash Codes:", hash_codes)
print("Selected Anchor Points Indices:", anchor_indices)

features extracted
anchor points done
anchor points generated
DONE :))) PLS FUCKING WORK
Generated Hash Codes: [[0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 [0 1 0 ... 1 1 1]
 ...
 [0 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 1]
 [0 1 0 ... 0 0 0]]
Selected Anchor Points Indices: [3316 9092  639  423 4298 9063  322 9059 5244 7472]


In [8]:
train = hash_codes[0:10000]
test = hash_codes[10000:]

In [9]:
def average_precision_at_k(retrieved_labels, true_label):
    """Calculates the average precision for a single query based on retrieved labels."""
    correct = 0
    precision_at_i = []
    
    for i in range(1, len(retrieved_labels) + 1):
        if retrieved_labels[i - 1] == true_label:
            correct += 1
            precision_at_i.append(correct / i)
    
    if len(precision_at_i) == 0:
        return 0
    return np.mean(precision_at_i)

def map_at_single_threshold(train_hash_codes, train_labels, test_hash_codes, test_labels, threshold):
    """Calculates MAP for a given distance threshold."""
    
    # Calculate the distance matrix between test and train hash codes
    distances = cdist(test_hash_codes, train_hash_codes, metric='cityblock')
    
    ap_scores = []
    
    for i in range(len(test_labels)):
        # Get distances and corresponding labels for the i-th test sample
        test_distances = distances[i]
        
        # Retrieve indices of training samples within the current threshold distance
        retrieved_indices = np.where(test_distances <= threshold)[0]
        
        if len(retrieved_indices) == 0:
            # If no neighbors are found within the distance threshold, skip this test sample
            continue
        
        # Get the labels of the retrieved neighbors
        retrieved_labels = train_labels[retrieved_indices]
        true_label = test_labels[i]
        
        # Compute average precision for the current test sample
        ap = average_precision_at_k(retrieved_labels, true_label)
        ap_scores.append(ap)
    
    # Compute mean average precision (MAP)
    if len(ap_scores) == 0:
        return 0  # No valid samples for this threshold
    return np.mean(ap_scores)

In [10]:
def precision_at_k(retrieved_labels, true_label, k):
    """Calculates Precision at K for a single query based on retrieved labels."""
    relevant_items = 0
    
    for i in range(min(k, len(retrieved_labels))):
        if retrieved_labels[i] == true_label:
            relevant_items += 1
    
    return relevant_items / k if k > 0 else 0

In [11]:
def evaluate_with_different_k_and_thresholds(train_hash_codes, train_labels, test_hash_codes, test_labels, k_values, thresholds):
    """Evaluates the image retrieval system using MAP and Precision at K for various thresholds and K values."""
    
    for threshold in thresholds:
        map_score = map_at_single_threshold(train_hash_codes, train_labels, test_hash_codes, test_labels, threshold)
        print(f"Mean Average Precision (MAP) at threshold {threshold}: {map_score:.4f}")
        
        for k in k_values:
            precision_scores = []
            distances = cdist(test_hash_codes, train_hash_codes, metric='cityblock')
            
            for i in range(len(test_labels)):
                test_distances = distances[i]
                retrieved_indices = np.argsort(test_distances)
                retrieved_labels = train_labels[retrieved_indices]
                precision = precision_at_k(retrieved_labels, test_labels[i], k)
                precision_scores.append(precision)
            
            mean_precision_at_k = np.mean(precision_scores) if precision_scores else 0
            print(f"Mean Precision at {k} (threshold {threshold}): {mean_precision_at_k:.4f}")

# Example usage:
k_values = [1, 5, 10, 15]  # You can choose different values for K
thresholds = [5, 10, 15]    # You can set various thresholds

evaluate_with_different_k_and_thresholds(train, y_train, test, y_test, k_values, thresholds)

Mean Average Precision (MAP) at threshold 5: 0.7241
Mean Precision at 1 (threshold 5): 0.8847
Mean Precision at 5 (threshold 5): 0.8764
Mean Precision at 10 (threshold 5): 0.8700
Mean Precision at 15 (threshold 5): 0.8636
Mean Average Precision (MAP) at threshold 10: 0.2564
Mean Precision at 1 (threshold 10): 0.8847
Mean Precision at 5 (threshold 10): 0.8764
Mean Precision at 10 (threshold 10): 0.8700
Mean Precision at 15 (threshold 10): 0.8636
Mean Average Precision (MAP) at threshold 15: 0.1103
Mean Precision at 1 (threshold 15): 0.8847
Mean Precision at 5 (threshold 15): 0.8764
Mean Precision at 10 (threshold 15): 0.8700
Mean Precision at 15 (threshold 15): 0.8636
